# Summarization (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

Installing libraries.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install huggingface_hub 
!pip install rouge
!pip install nltk

Fixes encoding error with colab.

In [112]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

Mount Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Setting up Git.

In [4]:
!git config --global user.email "ammar_amjad@ymail.com"
!git config --global user.name "Ammar-Amjad"

Logging into the Hugging Face Hub.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Setting up directories.

In [ ]:
# dir = "/content/drive/MyDrive/dataset/"
dir = "/content/"
train = dir + "urdu_train.jsonl" 
test = dir + "urdu_test.jsonl"
val = dir + "urdu_val.jsonl"

ntrain = dir + "/SmallerDataset/" + "urdu_train.csv"
ntest = dir + "/SmallerDataset/" + "urdu_test.csv"
nval = dir + "/SmallerDataset/" + "urdu_val.csv"

In [ ]:
import torch

In [92]:
from datasets import load_dataset, Dataset 
urdu_dataset = load_dataset('json', data_files={'train': train,
                                              'test': test,
                                              'val': val})

print(urdu_dataset)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 67665
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8458
    })
    val: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8458
    })
})


# New section

In [94]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['title']}'")
        print(f"'>> Summary: {example['summary']}'")

show_samples(urdu_dataset)


'>> Title: ’ادھوری نیند دفتر میں لڑائی اور خراب رویے کا سبب‘'
'>> Summary: راٹرڈیم سکول آف مینیجمینٹ کے محقیقین کے مطابق صرف ایک رات کی ادھوری نیند دفتر میں لڑائی اور خراب رویے کا سبب بن سکتی ہے۔'

'>> Title: کوہلو: راکٹ باری کے 9 واقعات'
'>> Summary: بلوچستان کے شہر کوہلو میں آج یوم آزادی پر نامعلوم افراد نے کم سے کم نو راکٹ داغے ہیں اور دو دھماکوں کی اطلاع موصول ہوئی ہے لیکن کسی قسم کا کوئی جانی نقصان نہیں ہوا ہے۔'

'>> Title: فائر بریگیڈ نے تاخیر کی، فیکٹری مالک کا الزام'
'>> Summary: کراچی میں آتشزدگی کا شکار بننے والی فیکٹری کے مالک نے واضح کیا ہے کہ واقعے والے روز فیکٹری کے دروازے بند نہیں تھے، انہوں نے فائر بریگیڈ پر غفلت کا الزام عائد کیاہے۔'


In [97]:
urdu_dataset.reset_format()

In [98]:
urdu_news = urdu_dataset

In [99]:
from datasets import concatenate_datasets, DatasetDict

urdu_news_dataset = DatasetDict()

for split in urdu_news.keys():
    urdu_news_dataset[split] = concatenate_datasets(
        [urdu_news[split]
        ]
    )
    urdu_news_dataset[split] = urdu_news_dataset[split].shuffle(seed=42)

show_samples(urdu_news_dataset)


'>> Title: پانچ کروڑ کہاں گئے؟'
'>> Summary: وانا کےمبینہ شدت پسندوں حاجی محمد عمر، حاجی شریف خان، مولوی محمد عباس اور جاوید خان کرمزخیل نے اس بیان پر شدید ردعمل کا اظہار کیا ہے کہ فوج کی جانب سے انہیں پانچ کروڑ روپے ادا کیے گئے ہیں۔'

'>> Title: ’مانچسٹر یونائیٹڈ کے سنہری دور کا اختتام ؟‘'
'>> Summary: امریکی یونیورسٹی مشی گن کے شعبۂ معاشیات کے پروفیسر اور فٹبال کی کتاب ’سوکرنومکس‘ کے مصنف سٹیون سائمنسکی کا کہنا ہے کے فٹبال کی دنیا کے تقریباً 90 فیصد مینیجرز کا اپنی ٹیم کی ہار جیت میں عمل دخل بہت کم ہوتا ہے۔'

'>> Title: سنوکر:صالح پاکستانی یا افغانی؟'
'>> Summary: سندھ ہائی کورٹ کے چیف جسٹس صبیح الدین احمد اور جسٹس گلزار احمد پر مشتمل بینچ نے سنوکر کے کھلاڑی صالح محمد کو اس ماہ کراچی میں ہونے والی ایشین سنوکر چیمپئن شپ میں حصہ لینے سے روکنے سے متعلق پٹیشن پر نوٹس جاری کرتے ہوئے سماعت چھ جون تک ملتوی کردی ہے۔'


Removing 1 word titles.

In [100]:
urdu_news_dataset = urdu_news_dataset.filter(lambda x: len(x["title"].split()) > 2)
print(urdu_news_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 67628
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8454
    })
    val: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 8454
    })
})


Loading Model.

In [101]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [102]:
inputs = tokenizer("مجھے ہنگر گیمز پڑھنا پسند تھا!")
inputs

{'input_ids': [4952, 21505, 259, 872, 162541, 12409, 50816, 1645, 21760, 1482, 6423, 1832, 9880, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [103]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁مج',
 'ھے',
 '▁',
 'ہ',
 'نگر',
 '▁گی',
 'مز',
 '▁پ',
 'ڑھ',
 'نا',
 '▁پس',
 'ند',
 '▁تھا',
 '!',
 '</s>']

Preprocessing

In [104]:
max_input_length = 11230 + 10
max_target_length = 36 + 2


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["summary"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [105]:
tokenized_datasets = urdu_news_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8454 [00:00<?, ? examples/s]

In [106]:
generated_summary = "پانچ کروڑ کہاں گئے؟"
reference_summary = "پاچ کروڑ کہاں گئے؟"

In [114]:
import evaluate

rouge_score = evaluate.load("rouge")

In [115]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}

In [117]:
from rouge import Rouge

generated_summary = "پانچ کروڑ کہاں گئے؟"
reference_summary = "پنچ کروڑ کہاں گئے؟"

def calc_Rouge(generated_summary, reference_summary):
  rouge = Rouge()
  scores = rouge.get_scores(generated_summary, reference_summary, avg=True)
  for s in scores:
    scores[s] = scores[s]['r']
  return scores
scores = calc_Rouge(generated_summary, reference_summary)
print("ROUGE Scores:", scores)


ROUGE Scores: {'rouge-1': 0.75, 'rouge-2': 0.6666666666666666, 'rouge-l': 0.75}


In [118]:
scores["rouge-1"]

0.75

In [120]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [121]:
from nltk.tokenize import sent_tokenize
import re

def extract_first_n_sentences(Summary, n=3):
    return Summary.split("۔")

def three_sentence_summary(Summary):
    return '\n'.join(extract_first_n_sentences(Summary)[:3])

print(three_sentence_summary(urdu_news_dataset["train"][1]["summary"]))

بلوچستان کے شہر کوہلو میں آج یوم آزادی پر نامعلوم افراد نے کم سے کم نو راکٹ داغے ہیں اور دو دھماکوں کی اطلاع موصول ہوئی ہے لیکن کسی قسم کا کوئی جانی نقصان نہیں ہوا ہے



In [122]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(Summary) for Summary in dataset["summary"]]
    if metric == "rouge":
      return calc_Rouge(summaries, dataset["title"])

In [123]:
import pandas as pd
metric = "rouge"
score = evaluate_baseline(urdu_news_dataset["val"], metric)
rouge_names = ["rouge-1", "rouge-2", "rouge-l"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge-1': 52.34, 'rouge-2': 22.35, 'rouge-l': 46.96}

In [124]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [125]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


Define Model Parameters

In [127]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [128]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = calc_Rouge(
        decoded_preds, decoded_labels
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [129]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [130]:
tokenized_datasets = tokenized_datasets.remove_columns(
    urdu_news_dataset["train"].column_names
)

In [131]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[   916,  13360,  44134,    633,  16887,   3904,  30589,    259,  72465,
           1956,  10785,   3643,    775,    548,  49243,   2154,    775,   8764,
          17629,    259,   8799,   3707,    259,   7123,    803,    858,  29193,
           9386,  20744,   1832,    259,  39853,    973,    259,    572,   9399,
           5989,   1159,  13906,   4283,   8794,    991,   1072,    259,  11481,
           2864,  16887,   2394,   4006,      1,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0],
        [  3923,  31655,   5322,    775,   2239,   7499,   1062,    872,   6495,
            973,   1424,   1956,    259,  71301,   6864,    406,   1197,   5738,
          20697,    259,  11890,   2138,   3980,   1295,   3980,   2859,    916,
          44964,    550,  33730,    991,   3612,   1159,   2858,  40365, 214536,
           1697,    803,  44234,   3433,  16445,  32825,    406,   1539,   3207,
           4159,    259,   92

In [132]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/content/mt5-small-finetuned-amazon-en-es is already a clone of https://huggingface.co/Ammar-Amjad/mt5-small-finetuned-amazon-en-es. Make sure you pull the latest changes with `repo.git_pull()`.


Train model

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,3.267200,2.166194,25.398100,10.816200,24.406100
2,2.549000,2.060939,26.463300,11.480900,25.486500
3,2.402200,1.996409,26.727900,11.587600,25.669000
4,2.314500,1.961013,26.898600,11.832100,25.901700


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
Several commits (2) will be pushed upstream.


Evaluation

In [48]:
trainer.evaluate()

{'eval_loss': 8.850305557250977,
 'eval_rouge-1': 6.8132,
 'eval_rouge-2': 3.3333,
 'eval_rouge-l': 6.8132,
 'eval_runtime': 0.4707,
 'eval_samples_per_second': 21.244,
 'eval_steps_per_second': 4.249,
 'epoch': 8.0}